# Pydantic Configurations

> - Documentation: https://pydantic-docs.helpmanual.io/
> - Configuration for Pydantic models: https://docs.pydantic.dev/latest/api/config/



```python
from pydantic import BaseModel, ConfigDict

class Model(BaseModel):
    model_config = ConfigDict()

```

## Handling Extra Fields

In [481]:
from pydantic import BaseModel, ConfigDict, ValidationError

class Model(BaseModel):
    field_1: int

In [482]:
display(Model.model_config)

{}

In [483]:
# This will not raise an error because extra fields are allowed and ignored by default
m1 = Model(field_1=1, field_2=2)
print(m1)
print(m1.model_fields)
print(m1.model_fields_set)

# See extra fields
print(m1.model_extra)

field_1=1
{'field_1': FieldInfo(annotation=int, required=True)}
{'field_1'}
None


__extra__: 

Whether to ignore, allow, or forbid extra attributes during model initialization. Defaults to 'ignore'.
You can configure how pydantic handles the attributes that are not defined in the model:

- `allow` - Allow any extra attributes.
- `forbid` - Forbid any extra attributes.
- `ignore` - Ignore any extra attributes.


Doc: https://docs.pydantic.dev/latest/api/config/#pydantic.config.ConfigDict.extra

In [484]:
class Model(BaseModel):
    model_config = ConfigDict(extra="forbid")

    field_1: int

In [485]:
display(Model.model_config)

{'extra': 'forbid'}

In [486]:
# This will raise an error because extra fields are forbidden
try:
    m2 = Model(field_1=1, field_2=2)
except ValidationError as e:
    print(e)

1 validation error for Model
field_2
  Extra inputs are not permitted [type=extra_forbidden, input_value=2, input_type=int]
    For further information visit https://errors.pydantic.dev/2.8/v/extra_forbidden


In [487]:
class Model(BaseModel):
    model_config = ConfigDict(extra="allow")

    field_1: int

In [488]:
display(Model.model_config)

{'extra': 'allow'}

In [489]:
# This will not raise an error because extra fields are allowed
m3 = Model(field_1=1, field_2=2)
display(m3)
print("model fields",m3.model_fields)
print("fields set", m3.model_fields_set)

# See extra fields
print("extra fields",m3.model_extra)

Model(field_1=1, field_2=2)

model fields {'field_1': FieldInfo(annotation=int, required=True)}
fields set {'field_1', 'field_2'}
extra fields {'field_2': 2}


## Strict and Lax Mode (Type Coercion)

In [490]:
class Model(BaseModel):
    field_1: str
    field_2: float
    field_3: list
    field_4: tuple

In [491]:
try:
    Model(field_1=9999, field_2=2, field_3=(1,2,3), field_4=[1,2,3])
except ValidationError as e:
    print(e)

1 validation error for Model
field_1
  Input should be a valid string [type=string_type, input_value=9999, input_type=int]
    For further information visit https://errors.pydantic.dev/2.8/v/string_type


In [492]:
Model(field_1="9999", field_2=2, field_3=(1,2,3), field_4=[1,2,3])

Model(field_1='9999', field_2=2.0, field_3=[1, 2, 3], field_4=(1, 2, 3))

In [493]:
# Set strict mode to True
class Model(BaseModel):
    model_config = ConfigDict(strict=True)

    field_1: str
    field_2: float
    field_3: list
    field_4: tuple

In [494]:
try:
    Model(field_1="9999", field_2=2, field_3=(1,2,3), field_4=[1,2,3])
except ValidationError as e:
    print(e)

2 validation errors for Model
field_3
  Input should be a valid list [type=list_type, input_value=(1, 2, 3), input_type=tuple]
    For further information visit https://errors.pydantic.dev/2.8/v/list_type
field_4
  Input should be a valid tuple [type=tuple_type, input_value=[1, 2, 3], input_type=list]
    For further information visit https://errors.pydantic.dev/2.8/v/tuple_type


https://docs.pydantic.dev/latest/concepts/conversion_table/#__tabbed_1_2

In [495]:
json_data = """
{
    "field_1": true,
    "field_2": 10,
    "field_3": 1,
    "field_4": null,
    "field_5": [1, 2, 3],
    "field_6": ["a", "b", "c"],
    "field_7": {"a": 1, "b": 2}
}
"""

In [496]:
class Model(BaseModel):
    model_config = ConfigDict(strict=True)

    field_1: bool
    field_2: float
    field_3: int
    field_4: str | None
    field_5: tuple[int, ...]
    field_6: set[str]
    field_7: dict

In [497]:
data = Model.model_validate_json(json_data)
display(data)

Model(field_1=True, field_2=10.0, field_3=1, field_4=None, field_5=(1, 2, 3), field_6={'a', 'c', 'b'}, field_7={'a': 1, 'b': 2})

In [498]:
json_data = """
{
    "field_1": true,
    "field_2": 10,
    "field_3": 1,
    "field_4": null,
    "field_5": [1, 2, 3.5],
    "field_6": ["a", "b", 100],
    "field_7": {"a": 1, "b": 2}
}
"""

try:
    Model.model_validate_json(json_data)
except ValidationError as ex:
    print(ex)

2 validation errors for Model
field_5.2
  Input should be a valid integer [type=int_type, input_value=3.5, input_type=float]
    For further information visit https://errors.pydantic.dev/2.8/v/int_type
field_6.2
  Input should be a valid string [type=string_type, input_value=100, input_type=int]
    For further information visit https://errors.pydantic.dev/2.8/v/string_type


## Validating Default Values

The default values of the fields are not validated by default. You can enable the validation of the default values by setting the `validate_assignment` to True.

```python
class Model(BaseModel):
    model_config = ConfigDict(validate_default=True)
```

In [499]:
# Default behavior
class Model(BaseModel):
    field_1: int = None
    field_2: str =  999

m = Model()
print(m)


field_1=None field_2=999


In [500]:
# This will raise an error because the default values are not valid
class Model(BaseModel):
    model_config = ConfigDict(validate_default=True)
    field_1: int = None
    field_2: str =  999

try:
    Model()
except ValidationError as ex:
    print(ex)


2 validation errors for Model
field_1
  Input should be a valid integer [type=int_type, input_value=None, input_type=NoneType]
    For further information visit https://errors.pydantic.dev/2.8/v/int_type
field_2
  Input should be a valid string [type=string_type, input_value=999, input_type=int]
    For further information visit https://errors.pydantic.dev/2.8/v/string_type


## Validating Assignment

By default, Pydantic does not validate the assignment of the values to the fields. You can enable the validation of the assignment by setting the `validate_assignment` to True.

```python
class Model(BaseModel):
    model_config = ConfigDict(validate_assignment=True)
```

In [501]:
# This will raise an error because the default values are not valid
class Model(BaseModel):
    field: int


m = Model(field=1)
m.field = "string 😔"
display(m)

Model(field='string 😔')

In [502]:
# This will raise an error because the default values are not valid
class Model(BaseModel):
    model_config = ConfigDict(validate_assignment=True)

    field: int


m = Model(field=1)

try:
    m.field = "string"
except ValidationError as ex:
    print(ex)


1 validation error for Model
field
  Input should be a valid integer, unable to parse string as an integer [type=int_parsing, input_value='string', input_type=str]
    For further information visit https://errors.pydantic.dev/2.8/v/int_parsing


# Mutability

In [503]:
class Model(BaseModel):
    model_config = ConfigDict(frozen=True)

    field: int

m = Model(field=1)
print(m)

field=1


In [504]:
# You cannot change the value of a field because the model is frozen
try:
    m.field = 2
except ValidationError as ex:
    print(ex)

1 validation error for Model
field
  Instance is frozen [type=frozen_instance, input_value=2, input_type=int]
    For further information visit https://errors.pydantic.dev/2.8/v/frozen_instance


## Standardizing String Fields



In [505]:
class Model(BaseModel):
    model_config = ConfigDict(str_strip_whitespace=True)

    field: str

m = Model(field="  hello  ")
display(m)

m1 = Model(field="  Python\n\t    ")
display(m1)

m2 = Model(field="\n\t  Python\n\t    ")
display(m2)

display(m1.field == m2.field)

Model(field='hello')

Model(field='Python')

Model(field='Python')

True

### Lowercase

In [506]:
class Model(BaseModel):
    model_config = ConfigDict(str_strip_whitespace=True, str_to_lower=True)

    field: str

m = Model(field="  HELLO  ")
display(m)

m1 = Model(field="  PythOn\n\t    ")
display(m1)

m2 = Model(field="\n\t  PYTHON\n\t    ")
display(m2)

display(m1.field == m2.field)

Model(field='hello')

Model(field='python')

Model(field='python')

True

### Uppercase

In [507]:
class Model(BaseModel):
    model_config = ConfigDict(str_strip_whitespace=True, str_to_upper=True)

    field: str

m = Model(field="  HeLLo  ")
display(m)

m1 = Model(field="  PythOn\n\t    ")
display(m1)

m2 = Model(field="\n\t  PYTHON\n\t    ")
display(m2)

display(m1.field == m2.field)

Model(field='HELLO')

Model(field='PYTHON')

Model(field='PYTHON')

True

# Handling Python Enum


In [508]:
from enum import Enum

class Color(Enum):
    red = "Red"
    green = "Green"
    blue = "Blue"
    orange = "Orange"
    yellow = "Yellow"
    cyan = "Cyan"
    white = "White"
    black = "Black"

In [509]:
display(Color.red.value)
display(Color.red.name)

'Red'

'red'

In [510]:
class Model(BaseModel):
    color: Color

Model(color=Color.red)

Model(color=<Color.red: 'Red'>)

In [511]:
data_color = """
{
    "color": "Red"
}
"""
Model.model_validate_json(data_color)

Model(color=<Color.red: 'Red'>)

In [512]:
data_color_invalid = """
{
    "color": "Magenta"
}
"""
try:
    Model.model_validate_json(data_color_invalid)
except ValidationError as ex:
    print(ex)

1 validation error for Model
color
  Input should be 'Red', 'Green', 'Blue', 'Orange', 'Yellow', 'Cyan', 'White' or 'Black' [type=enum, input_value='Magenta', input_type=str]
    For further information visit https://errors.pydantic.dev/2.8/v/enum


In [513]:
m = Model.model_validate_json(data_color)
display("Dict:", m.model_dump())  # This will return a dictionary with the enum
display("Json:", m.model_dump_json()) # This will return a json string with the enum value

'Dict:'

{'color': <Color.red: 'Red'>}

'Json:'

'{"color":"Red"}'

We can use the `use_enum_values` parameter to return the enum value instead of the enum object in the dump methods


In [514]:

class Model(BaseModel):
    model_config = ConfigDict(use_enum_values=True)
    color: Color

Model(color=Color.red)

m = Model.model_validate_json(data_color)
display("Dict:", m.model_dump()) # This will return a dictionary with the enum value
display("Json:", m.model_dump_json()) # This will return a json string with the enum value

'Dict:'

{'color': 'Red'}

'Json:'

'{"color":"Red"}'

### Caveats
Because pydantic does not validate default values, even if you set `use_enum_values` to True, the default value will be the enum object. 

In [515]:
class Model(BaseModel):
    model_config = ConfigDict(use_enum_values=True) # The model is set to use the enum values
    color: Color = Color.red

Model() # The default value is used, and it is should be the enum value "Red"

Model(color=<Color.red: 'Red'>)

In [516]:
class Model(BaseModel):
    model_config = ConfigDict(use_enum_values=True, validate_default=True)
    color: Color = Color.red

Model()

Model(color='Red')